In [ ]:
import pandas as pd
data_url = "https://open.jacksonms.gov/datastore/odata3.0/2fd76604-4378-4e19-9f0f-1bf220825c97?$format=json"


# retrieve json formatted data from url & import as dataframe
df = pd.read_json(data_url, orient="records")

print(df)

In [ ]:
# manipulate or clean data - nested json to column data
clean_df = df.value.apply(pd.Series)

print(clean_df)

clean_df = clean_df.dropna()

# Wrte new data frame to csv
clean_df.to_csv('clean_data.csv', index=False)

In [6]:
clean_df['Attendance'].median()

1600.0

In [7]:
concert_data = clean_df.loc[clean_df['Type'] == "Concert"]
print(concert_data)

    Attendance     Revenue                                      Name of Event  \
0         1000   2550.0000  R&B Concert featuring Tre' Williams, Calvin Ri...   
1         3800   5100.0000                                            Rave On   
14        2360   1550.0000                               Robin Thicke Concert   
17        2519   5895.7700                            I Can't Stop Loving You   
43         800   1250.0000                             Congregation Gone Wild   
44        2000   2500.0000                                 Trey Songz Concert   
49        1438   2500.0000                             Kevin Hart Comedy Show   
63           0      0.0000                                 El DeBarge Concert   
70        2350   2500.0000                   Third Day and Tenth Avenue North   
76        1855   2550.0000                  Fantasia and Urban Mystic Concert   
79        1901   2600.0000                             Jackson Browne Concert   
98        1033   2550.0000  

In [10]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import Set3_10
from bokeh.transform import factor_cmap

# Output the plot to the notebook
# To save as html, import ouput_file from bokeh.plottig import output_file
# Use the code: output_file("revenue_barplot.html")
output_notebook()

# Convert Revenue column to floats to sum
clean_df['Revenue'] = clean_df['Revenue'].astype(float)
# Group data by Date
grouped = clean_df.groupby('Date')['Revenue'].sum()

print(grouped)

Loading BokehJS ...

Date
2009     16050.00
2010     93638.49
2011     92237.32
2012     75251.00
2013     90547.43
2014     67016.00
2015    155918.31
2016    214761.20
2017    130920.18
2018    141346.25
Name: Revenue, dtype: float64


In [11]:
source = ColumnDataSource(pd.DataFrame(grouped))
dates = source.data['Date'].tolist()
p = figure(x_range=dates)
color_map = factor_cmap(field_name='Date', palette=Set3_10, factors=dates)

p.vbar(x='Date', top='Revenue', source=source, width=0.70, color=color_map)

p.title.text ='Jackson Event Revenue by Year'
p.xaxis.axis_label = 'Year'
p.yaxis.axis_label = 'Revenue'

show(p)